*Copyright (c) Microsoft Corporation. All rights reserved.*

*Licensed under the MIT License.*

# Text Classification of MultiNLI Sentences using XLNet

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../")
import os
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from utils_nlp.dataset.multinli import load_pandas_df
from utils_nlp.eval.classification import eval_classification
from utils_nlp.common.timer import Timer
from utils_nlp.models.xlnet.common import Language, Tokenizer
from utils_nlp.models.xlnet.sequence_classification import XLNetSequenceClassifier
from utils_nlp.models.xlnet.utils import generate_confusion_matrix
from utils_nlp.models.xlnet.common import log_xlnet_params
import mlflow
import datetime

## Introduction
In this notebook, we fine-tune and evaluate a pretrained [XLNet](https://arxiv.org/abs/1906.08237) model on a subset of the [MultiNLI](https://www.nyu.edu/projects/bowman/multinli/) dataset.

We use a [sequence classifier](../../utils_nlp/xlnet/sequence_classification.py) that wraps [Hugging Face's PyTorch implementation](https://github.com/huggingface/pytorch-transformers) of CMU and Google's [XLNet](https://github.com/zihangdai/xlnet).

In [3]:
DATA_FOLDER = "../../../temp"
XLNET_CACHE_DIR="../../../temp"
LANGUAGE = Language.ENGLISHCASED
MAX_SEQ_LENGTH = 128
BATCH_SIZE = 4
NUM_GPUS = 1
NUM_EPOCHS = 3
TRAIN_SIZE = 0.6
LABEL_COL = "genre"
TEXT_COL = "sentence1"
WEIGHT_DECAY = 0.0
WARMUP_STEPS = 0

### Hyperparamters to tune
MAX_SEQ_LENGTH = 128
LEARNING_RATE = 5e-5
ADAM_EPSILON = 1e-8

DEBUG = True
LOGGING_STEPS = 10
mlflow.start_run(run_name = datetime.datetime.now())
log_xlnet_params(locals())

## Read Dataset
We start by loading a subset of the data. The following function also downloads and extracts the files, if they don't exist in the data folder.

The MultiNLI dataset is mainly used for natural language inference (NLI) tasks, where the inputs are sentence pairs and the labels are entailment indicators. The sentence pairs are also classified into *genres* that allow for more coverage and better evaluation of NLI models.

For our classification task, we use the first sentence only as the text input, and the corresponding genre as the label. We select the examples corresponding to one of the entailment labels (*neutral* in this case) to avoid duplicate rows, as the sentences are not unique, whereas the sentence pairs are.

In [4]:
df = load_pandas_df(DATA_FOLDER, "train")
df = df[df["gold_label"]=="neutral"]  # get unique sentences

if DEBUG:
    inds = random.sample(range(len(df.index)), 1000)
    df = df.iloc[inds]

In [5]:
df.head()

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
286510,[neutral],fiction,neutral,87443n,87443,"""We'll have a delivery of remounts to make to...",( `` ( We ( ( 'll ( have ( ( ( a delivery ) ( ...,(ROOT (S (`` ``) (NP (PRP We)) (VP (MD 'll) (V...,The remounts were ordered by those of the camp.,( ( The remounts ) ( ( were ( ordered ( by ( t...,(ROOT (S (NP (DT The) (NNS remounts)) (VP (VBD...
254119,[neutral],government,neutral,143239n,143239,(iii) The regulations under clause (i) may inc...,( ( ( ( -LRB- ( iii -RRB- ) ) ( ( The regulati...,(ROOT (S (NP (NP (NP (-LRB- -LRB-) (NN iii) (-...,Emissions reductions are actively being pursued.,( ( Emissions reductions ) ( ( ( are actively ...,(ROOT (S (NP (JJ Emissions) (NNS reductions)) ...
232736,[neutral],government,neutral,91574n,91574,"America needs a clean, secure, affordable, rel...","( America ( ( ( needs ( ( a ( ( clean ( , ( se...",(ROOT (S (NP (NNP America)) (VP (VBZ needs) (N...,Many believe that without changes to the energ...,( Many ( ( believe ( that ( ( without ( change...,(ROOT (S (NP (DT Many)) (VP (VBP believe) (SBA...
281705,[neutral],slate,neutral,100430n,100430,"(Impeachment, of course, is hardly the sole ca...","( -LRB- ( ( ( ( Impeachment , ) ( of course ) ...",(ROOT (S (-LRB- -LRB-) (NP (NP (NNP Impeachmen...,He was paralyzed after he fell down some stairs.,( He ( ( was ( paralyzed ( after ( he ( ( fell...,(ROOT (S (NP (PRP He)) (VP (VBD was) (VP (VBN ...
344823,[neutral],travel,neutral,142225n,142225,Next door on the busy pedestrian street of Cor...,( ( ( Next door ) ( on ( ( the ( busy ( pedest...,(ROOT (S (NP (NP (JJ Next) (NN door)) (PP (IN ...,The 15-century hall and chapel is open for tou...,( ( ( ( The ( 15-century hall ) ) and ) chapel...,(ROOT (S (NP (NP (DT The) (JJ 15-century) (NN ...


The examples in the dataset are grouped into 5 genres:

In [6]:
df[LABEL_COL].value_counts()

government    214
fiction       211
telephone     201
travel        200
slate         174
Name: genre, dtype: int64

We split the data for training and testing, and encode the class labels:

In [7]:
# split
df_train, df_test = train_test_split(df, train_size = TRAIN_SIZE)

# encode labels
label_encoder = LabelEncoder()
labels_train = label_encoder.fit_transform(df_train[LABEL_COL])
labels_test = label_encoder.transform(df_test[LABEL_COL])
label_list = label_encoder.classes_

num_labels = len(np.unique(labels_train))

In [8]:
print("Number of unique labels: {}".format(num_labels))
print("Number of training examples: {}".format(df_train.shape[0]))
print("Number of testing examples: {}".format(df_test.shape[0]))

Number of unique labels: 5
Number of training examples: 600
Number of testing examples: 400


## Tokenize and Preprocess
Before training, we tokenize the text documents and convert them to lists of tokens. The following steps instantiate a XLNet tokenizer given the language, and tokenize the text of the training and testing sets.

We perform the following preprocessing steps in the cell below:
- Convert the tokens into token indices corresponding to the XLNet-base tokenizer's vocabulary
- Add the special tokens [CLS] and [SEP] to mark the end of a sentence
- Pad or truncate the token lists to the specified max length
- Return id lists that indicate which word the tokens map to
- Return mask lists that indicate paddings' positions
- Return segment type id lists that indicates which segment each the tokens belongs to

*See the pytorch-transformer [implementation](https://github.com/huggingface/pytorch-transformers/blob/master/examples/utils_glue.py) for more information on XLNet's input format.*

In [9]:
tokenizer = Tokenizer(LANGUAGE, cache_dir=XLNET_CACHE_DIR)

train_input_ids, train_input_mask, train_segment_ids = tokenizer.preprocess_classification_tokens(list(df_train[TEXT_COL]), MAX_SEQ_LENGTH)
test_input_ids, test_input_mask, test_segment_ids = tokenizer.preprocess_classification_tokens(list(df_test[TEXT_COL]), MAX_SEQ_LENGTH)

## Create Model
Next, we create a sequence classifier that loads a pre-trained XLNet model, given the language and number of labels.

In [10]:
classifier = XLNetSequenceClassifier(
    language=LANGUAGE,
    num_labels=num_labels,
    cache_dir=XLNET_CACHE_DIR,
    num_gpus=NUM_GPUS,        
    num_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE
)

## Train
We train the classifier using the training examples. This involves fine-tuning the XLNet Transformer and learning a linear classification layer on top of that:

In [11]:
with Timer() as t:
    classifier.fit(
        token_ids=train_input_ids,
        input_mask=train_input_mask,
        token_type_ids=train_segment_ids,
        labels=labels_train,  
        verbose=True,
        logging_steps = LOGGING_STEPS,
    )    
print("[Training time: {:.3f} hrs]".format(t.interval / 3600))

Iteration:   1%|          | 1/150 [00:00<01:22,  1.81it/s]

epoch:1/3; batch:1->16/150; average training loss:1.727106


Iteration:  11%|█▏        | 17/150 [00:06<00:51,  2.56it/s]

epoch:1/3; batch:17->32/150; average training loss:1.785313


Iteration:  22%|██▏       | 33/150 [00:13<00:45,  2.55it/s]

epoch:1/3; batch:33->48/150; average training loss:1.698002


Iteration:  33%|███▎      | 49/150 [00:19<00:39,  2.55it/s]

epoch:1/3; batch:49->64/150; average training loss:1.571066


Iteration:  34%|███▍      | 51/150 [00:20<00:38,  2.57it/s]

KeyboardInterrupt: 

## Score
We score the test set using the trained classifier:

In [ ]:
preds = classifier.predict(
    token_ids=test_input_ids,
    input_mask=test_input_mask,
    token_type_ids=test_segment_ids,
    num_gpus=NUM_GPUS,
    batch_size=BATCH_SIZE,
    probabilities=False
)

## Evaluate Results
Finally, we compute the accuracy, precision, recall, and F1 metrics of the evaluation on the test set.

In [ ]:
cls_report = classification_report(labels_test, preds, target_names=label_encoder.classes_)
print(cls_report)

### Generate Confusion Matrix using Seaborn

In [ ]:
generate_confusion_matrix(labels_test,preds,label_encoder.classes_)

In [ ]:
mlflow.end_run()